### 1. Tạo pertured data và lưu giống data cũ (cal_metric.py)
    1. Tạo ra câu pertured va convert json (pertured_folder/masked_data_json)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. ghép label (từ 2.2) của câu gốc với câu mask
    4. đưa json (masked data 0.1) vào model lấy logit, prediction 
### 3. lấy được index của chỗ arg bị thay đổi của dự đoán sau khi masked data 
### 4. Tính inf 

## Step 1

In [2]:
import json
import os
import torch
import sys
from tqdm import tqdm
from mlm_utils.model_utils import TOKENIZER
from mlm_utils.transform_func import get_files, encode_text
from mlm_utils.pertured_dataset import PerturedDataset
from pathlib import Path
# sys.path.append('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/')
# from SRL.model import multiTaskModel

### 1.1

In [5]:
def read_data(readPath):
    
    with open(readPath, 'r', encoding = 'utf-8') as file:
        taskData = []
        for i, line in enumerate(file):
            sample = json.loads(line)
            taskData.append(sample)
            
    return taskData

def generate_data_and_write_json(dataDir, wriDir, lableDir):
    
    def gen_data_line(line):
        
        origin_sen = TOKENIZER.decode(torch.tensor(line['origin_id']), skip_special_tokens = True)

        new_word_list_set = set([line['cos_neg'], line['cos_0'], line['cos_module_neg'], line['cos_module_0']])

        masked_data_set = list(map(lambda x: origin_sen.replace(line['masked_word'], x, 1), new_word_list_set))
        return line['origin_id'], str(masked_data_set)
    
    files_data = get_files(dataDir)
    files_label = get_files(lableDir)
    
    
    for data_f, label_f in tqdm(zip(files_data, files_label)):
        if data_f == 'catalyse_full.json':
            data = read_data(dataDir + data_f)
            label_data = read_data(lableDir + label_f)
        
            masked_data_file = list(map(lambda x: gen_data_line(x), data))

            tmp_list = []
            # lay label giua pertured data voi coNLL_tsv_json      
            print("Process file", data_f)
            for idx, (origin_id, pertured_text) in enumerate(masked_data_file):
                flat = 0
                for line in label_data:
                    if list(filter(None, origin_id)) == list(filter(None, line['token_id'])):
                        flat = 1
                        tmp_list.append((origin_id, pertured_text, line['label']))
                    else: 
                        continue
                if flat == 0:
                    print("origin id", origin_id)
                    print("TOKENIZER", TOKENIZER.decode(torch.tensor(origin_id), skip_special_tokens = True))
                    break
                
            write_file = "pertured_data_{}".format(data_f)

            with open(os.path.join(wriDir, write_file) , 'w') as f:
                for idx, (origin_id, pertured_text, label) in enumerate(tmp_list):
                    encoded_text = encode_text(pertured_text)
                    sample = {
                        'uid': idx,
                        'origin_id': origin_id,
                        'label': label,
                        'token_id': encoded_text['input_ids'][0].tolist(),
                        'type_id': encoded_text['token_type_ids'][0].tolist(),
                        'mask': encoded_text['attention_mask'][0].tolist()
                    }
                    f.write('{}\n'.format(json.dumps(sample)))
            print("Done", data_f)
        

In [6]:
labelDir = './data_mlm/process_folder/coNLL_tsv_json/dmis-lab/biobert-base-cased-v1.2_prepared_data/'
generate_data_and_write_json(dataDir = './data_mlm/process_folder/pertured_data/', wriDir = './data_mlm/pertured_folder/masked_data_json', lableDir=labelDir)

0it [00:00, ?it/s]

Process file catalyse_full.json
origin id [101, 173, 1605, 1119, 9538, 8830, 1132, 1682, 1106, 5855, 17449, 3171, 1103, 8362, 11129, 1158, 1104, 173, 1605, 1118, 26499, 1158, 1103, 9986, 22587, 1206, 13185, 7616, 1112, 1152, 5070, 1373, 1103, 173, 1605, 15947, 1107, 170, 15281, 4633, 113, 1719, 2588, 1137, 4389, 114, 117, 1105, 1138, 1151, 2602, 1106, 1129, 2017, 1107, 1242, 6818, 14391, 5669, 117, 1259, 25544, 117, 15416, 117, 173, 1605, 6949, 1105, 5179, 113, 1111, 3761, 1267, 1745, 117, 1820, 132, 5285, 3084, 2393, 119, 117, 1772, 102]
TOKENIZER dna helicases are able to catalyze the unwinding of dna by disrupting the hydrogen bonding between paired bases as they progress along the dna strand in a polar fashion ( either 35 or 53 ), and have been shown to be involved in many essential cellular processes, including replication, transcription, dna repair and translation ( for reviews see west, 1996 ; bird et al., 1998


35it [00:00, 39.54it/s]

Done catalyse_full.json


3

In [9]:
lisst = ['dna', 'helicases', 'are', 'able', 'to', 'catalyze', 'the', 'unwinding', 'of', 'dna', 'by', 'disrupting', 'the', 'hydrogen', 'bonding', 'between', 'paired', 'bases', 'as', 'they', 'progress', 'along', 'the', 'dna', 'strand', 'in', 'a', 'polar', 'fashion', '(', 'either', '35', 'or', '53', ')', ',', 'and', 'have', 'been', 'shown', 'to', 'be', 'involved', 'in', 'many', 'essential', 'cellular', 'processes', ',', 'including', 'replication', ',', 'transcription', ',', 'dna', 'repair', 'and', 'translation', '(', 'for', 'reviews', 'see', 'west', ',', '1996', ';', 'bird', 'et', 'al', '.', ',', '1998', ';', 'lohman', 'et', 'al', '.', ',', '1998', ';', 'hall', 'and', 'matson', ',', '1999', ')', '.']
print(len(TOKENIZER.encode(" ".join(lisst), add_special_tokens = True)))

102


In [10]:
print(len([101, 1649, 117, 1103, 17019, 3943, 1144, 1151, 5855, 17449, 5591, 1118, 1231, 1643, 1559, 1604, 1105, 1231, 1643, 1545, 1604, 1146, 1106, 6087, 118, 11203, 1750, 19723, 117, 1780, 1152, 185, 13159, 1279, 1103, 2072, 1231, 1643, 1571, 1477, 1105, 1231, 1643, 12882, 19350, 4001, 117, 3569, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

85


In [29]:
t = torch.tensor([101, 173, 1605, 1119, 9538, 8830, 1132, 1682, 1106, 5855, 17449, 3171, 1103, 8362, 11129, 1158, 1104, 173, 1605, 1118, 26499, 1158, 1103, 9986, 22587, 1206, 13185, 7616, 1112, 1152, 5070, 1373, 1103, 173, 1605, 15947, 1107, 170, 15281, 4633, 113, 1719, 2588, 1137, 4389, 114, 117, 1105, 1138, 1151, 2602, 1106, 1129, 2017, 1107, 1242, 6818, 14391, 5669, 117, 1259, 25544, 117, 15416, 117, 173, 1605, 6949, 1105, 5179, 113, 1111, 3761, 1267, 1745, 117, 1820, 132, 5285, 3084, 2393, 119, 117, 1772, 102])
print(len(t))
print(TOKENIZER.decode(t, skip_special_tokens=True))

85
dna helicases are able to catalyze the unwinding of dna by disrupting the hydrogen bonding between paired bases as they progress along the dna strand in a polar fashion ( either 35 or 53 ), and have been shown to be involved in many essential cellular processes, including replication, transcription, dna repair and translation ( for reviews see west, 1996 ; bird et al., 1998


### 1.2

In [2]:

writeDir = Path('./data_mlm/pertured_folder/masked_data_json')
dataset = PerturedDataset(
        data_path=writeDir,
        file_name='pertured_data_abolish_full.json',
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

dataloader = dataset.generate_batches(
            dataset= dataset,
            batch_size=32)

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_params(model_file):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Load finetuned model
    loadedDict = torch.load(model_file, map_location=torch.device(device))

    taskParams = loadedDict['task_params']
    allParams = {}
    allParams['task_params'] = taskParams
    allParams['gpu'] = torch.cuda.is_available()

    # dummy values
    allParams['num_train_steps'] = 10
    allParams['warmup_steps'] = 0
    allParams['learning_rate'] = 2e-5
    allParams['epsilon'] = 1e-8

    return allParams, loadedDict      

In [4]:
import torch.nn as nn
import numpy as np

model_file = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/output/multi_task_model_9_13050.pt')
allParams, loadedDict = load_params(model_file)
model = multiTaskModel(allParams)
model.load_multi_task_model(loadedDict)
predicate_file = 'mlm_abolish_full.json'


In [10]:
import pandas as pd

def get_pred_score(dataloader, model):
    allPreds = []
    allScores = []
    allLogits = []
    for batch in tqdm(dataloader, total = len(dataloader)):
        batch = tuple(t.to(device) if isinstance(t, torch.Tensor) else t for t in batch)

        uid, origin_id, token_id, type_id, mask = batch
        with torch.no_grad():
            outputs_model, logits = model.network(token_id, type_id, mask, 0, 'conllsrl')
            
            outLogitsSoftmax = nn.functional.softmax(logits, dim = 2).data.cpu().numpy()
            outLogitsSigmoid = nn.functional.sigmoid(logits).data.cpu().numpy()
            #shape of logits now (batchSize, maxSeqLen, classNum: 15)
            
            predicted = np.argmax(outLogitsSoftmax, axis = 2)
            # here in score, we only want to give out the score of the class of tag, which is maximum
            predScore = np.max(outLogitsSigmoid, axis = 2).tolist()
            
            #shape of predicted now (batchSize, maxSeqLen)
            # logger.debug("Final Predictions shape after argmx: {}".format(predicted.shape))
            predicted = predicted.tolist()
        
            # get the attention masks, we need to discard the predictions made for extra padding
            
            predictedTags = []
            predScoreTags = []
            
            if mask is not None:
                #shape of attention Masks (batchSize, maxSeqLen)
                actualLengths = mask.cpu().numpy().sum(axis = 1).tolist()
            
                for i, (pred, sc) in enumerate(zip(predicted, predScore)):
                    predictedTags.append( pred[:actualLengths[i]] )
                    predScoreTags.append( sc[:actualLengths[i]])
    
            else:
                predictedTags = predicted
                predScoreTags = predScore
            
            allPreds.append(predictedTags)  
            allScores.append(predScoreTags)  
            allLogits.append(outLogitsSoftmax)
    # flatten allPreds, allScores
    allPreds = [item for sublist in allPreds for item in sublist]
    allScores = [item for sublist in allScores for item in sublist]
    return allPreds, allScores      


# def get_origin_labels(labelDir, perturedDir):
#     # lấy label thông qua origin_id của coNLL_tsv_json voi masked_data_json
#     label_file = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/process_folder/coNLL_tsv_json/dmis-lab/biobert-base-cased-v1.2_prepared_data/ner_conll_format_abolish_full.json'
#     perture_file = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/pertured_folder/masked_data_json/pertured_data_abolish_full.json'
    
#     with open(label_file, 'w') as label_f:
        
    
def write_pred_label(allPreds, allScores, labelDir, allParams, wrtFile):
    predicate_file = 'mlm_abolish_full.json'
    
    allLabels = []
    allIds = []
    
    
    with open(labelDir + predicate_file, 'r') as label_f:
        for i, line in enumerate(label_f):
            sample = json.loads(line)   
            allLabels.append(sample['label'])
            allIds.append(sample['uid'])
    labMap = allParams['task_params'].labelMap['conllsrl']
    
    labMapRevN = {v:k for k,v in labMap.items()}

    for j, (p, l) in enumerate(zip(allPreds[i], allLabels[i])):
        allLabels[i][j] = l[:len(p)]
        allPreds[i][j] = [labMapRevN[int(ele)] for ele in p]
        allLabels[i][j] = [labMapRevN[int(ele)] for ele in allLabels[i][j]]
    #allPreds[i] = [ [ labMapRev[int(p)] for p in pp ] for pp in allPreds[i] ]
    #allLabels[i] = [ [labMapRev[int(l)] for l in ll] for ll in allLabels[i] ]

    newPreds = []
    newLabels = []
    newScores = []
    for m, samp in enumerate(allLabels[i]):
        Preds = []
        Labels = []
        Scores = []
        for n, ele in enumerate(samp):
            #print(ele)
            if ele != '[CLS]' and ele != '[SEP]' and ele != 'X':
                #print('inside')
                Preds.append(allPreds[i][m][n])
                Labels.append(ele)
                Scores.append(allScores[i][m][n])
                
        newPreds.append(Preds)
        newLabels.append(Labels)
        newScores.append(Scores)
    
    allLabels[i] = newLabels
    allPreds[i] = newPreds
    allScores[i] = newScores
    for i in range(len(allPreds)):
        if allPreds[i] == []:
            continue
       
        df = pd.DataFrame({"uid" : allIds[i], "prediction" : allPreds[i], "label" : allLabels[i]})
        
        df.to_csv(wrtFile, sep = "\t", index = False)
    

In [12]:
# labelDir ='../MLM/data_mlm/process_folder/mlm_output/'
labelDir = '../MLM/data_mlm/process_folder/coNLL_tsv_json/dmis-lab/biobert-base-cased-v1.2_prepared_data/ner_conll_format_abolish_full.json'
allPreds, allScores = get_pred_score(dataloader, model)
wrtFile = "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/pertured_data_prediction.json"
write_pred_label(allPreds, allScores, labelDir, allParams, wrtFile)

100%|██████████| 32/32 [02:01<00:00,  3.79s/it]


KeyError: 'labels'

## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [ ]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 50